In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from PIL import Image
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class CTScanDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label, category in enumerate(["Normal", "Covid"]):
            category_path = os.path.join(root_dir, category)
            for img_name in os.listdir(category_path):
                self.image_paths.append(os.path.join(category_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CTScanDataset(root_dir='/Users/zankhanamehta/Documents/NEU/SML/Group project/Train', transform=transform)
val_dataset   = CTScanDataset(root_dir='/Users/zankhanamehta/Documents/NEU/SML/Group project/Val/',   transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # First two convolutional layers (original)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Additional convolutional layers
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        
        # Pooling and fully connected layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 14 * 14, 128)  # Adjust input size for new layers
        self.fc2 = nn.Linear(128, 2)  # Binary classification
        
        # Activation and dropout
        self.activation = nn.ReLU() 
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Pass through convolutional layers with activation and pooling
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = self.pool(self.activation(self.conv4(x)))

        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)
        x = self.dropout(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x

In [11]:
class MultiModel(nn.Module):
    def __init__(self, custom_cnn, densenet):
        super(MultiModel, self).__init__()
        self.custom_cnn = custom_cnn
        self.densenet = densenet
        # Freeze densenet layers
        for param in self.densenet.parameters():
            param.requires_grad = False

        # Output layers to combine features
        self.fc = nn.Linear(2 * 2, 2)  # Assuming binary classification

    def forward(self, x):
        custom_output = self.custom_cnn(x)
        densenet_output = self.densenet(x)
        combined_output = torch.cat((custom_output, densenet_output), dim=1)
        final_output = self.fc(combined_output)
        return final_output


In [13]:
custom_cnn = CustomCNN()
densenet = models.densenet121(pretrained=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 2)

# Load the trained weights
custom_cnn.load_state_dict(torch.load("covid_ct_model.pth"))
densenet.load_state_dict(torch.load("densenet_model.pth"))

# Create the multi-model
model = MultiModel(custom_cnn, densenet).to(device)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/var/folders/pc/65k66fpd65940gkc6sklbn3m0000gn/T/ipykernel_3808/1364189758.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/py

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch [1/10], Loss: 0.9378, Accuracy: 60.22%
Epoch [2/10], Loss: 0.4325, Accuracy: 83.59%
Epoch [3/10], Loss: 0.2730, Accuracy: 90.11%
Epoch [4/10], Loss: 0.1793, Accuracy: 95.87%
Epoch [5/10], Loss: 0.1145, Accuracy: 97.17%
Epoch [6/10], Loss: 0.0902, Accuracy: 98.70%
Epoch [7/10], Loss: 0.0978, Accuracy: 98.04%
Epoch [8/10], Loss: 0.0367, Accuracy: 99.24%
Epoch [9/10], Loss: 0.0277, Accuracy: 99.13%
Epoch [10/10], Loss: 0.0357, Accuracy: 99.13%


In [19]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 60.78%
